<a href="https://colab.research.google.com/github/Aman1647/Topic-Modelling-on-News-Articles/blob/main/Topic_Modelling_on_News_Articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

### Know Your Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display  # interactive display
from tqdm import tqdm   #progress bar of execution
from collections import Counter  # when u want to iterate over something and keep a count of that
import ast

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
import os                                                                        # for listing files in given directory

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

import spacy
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag, word_tokenize, RegexpParser

from sklearn.feature_extraction.text import CountVectorizer     # to create a dict --> how many times which word has occured in the document
from textblob import TextBlob
import scipy.stats as stats

from wordcloud import WordCloud

from sklearn.decomposition import TruncatedSVD              # Singular value decomposition
from sklearn.decomposition import LatentDirichletAllocation  
from sklearn.manifold import TSNE    # similar to PCA --> used for dimensionality reduction

#output_notebook()

%matplotlib inline

#### Installing packages

In [ ]:
!pip install contractions                                                        # for removing contractions
!pip install pyLDAvis  

In [ ]:
path = '/content/drive/MyDrive/Almabetter/Capstone Projects/Unsupervised Learning/Topic Modelling/bbc/'
folders = ["business", "entertainment", "politics", "sport", "tech"]

In [ ]:
news = []                                                                        
topics = []

for i in folders:
  lst_file_names = os.listdir(path+i)                                            # list of file names present in directory bbc
  for txt_files in lst_file_names:
    txt_path = path + i+ '/'+ txt_files                                          # exact path of all text files
    with open(txt_path, 'rb') as f:                                              # open a binary file
      text = f.read()                                                            # read all lines
      news.append(text)                                                          # append text files 
      topics.append(i)                                                           # append topics 


In [ ]:
# creating a dataframe of news & type
df = pd.DataFrame()
df['News_text'] = news
df['type'] = topics

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

### Data Wrangling

In [ ]:
# finding length of each news
df['length'] = df['News_text'].apply(len)

In [ ]:
# finding word countof each news
df['word_count'] = df['News_text'].apply(lambda x:len(str(x).split(" ")))

In [ ]:
df.head()

### EDA

In [ ]:
bbc_df = df.copy()

In [ ]:
topic_count = bbc_df['type'].value_counts()
topic_count

In [ ]:
plt.figure(figsize = (10, 8))
plt.pie(topic_count , labels = topic_count.index, autopct = '%0.2f%%' )
plt.title("Topic Distribution", size=15)
plt.plot()

In [ ]:
# length of news in each type
plt.figure(figsize = (10, 8))
sns.barplot(x= bbc_df['type'], y= bbc_df['length'])
plt.title('Length of News in each type', size=15)
plt.xlabel('Type')
plt.ylabel("Length of news")
plt.plot()

In [ ]:
# length of news in each type
plt.figure(figsize = (10, 8))
sns.barplot(x= bbc_df['type'], y= bbc_df['word_count'])
plt.title('Word count of News in each type', size=15)
plt.xlabel('Type')
plt.ylabel("Word count of news")
plt.plot()

### Handling Missing & Duplicate values

In [ ]:
# dropping duplicate values
bbc_df.drop_duplicates(inplace=True)

### Text Pre-processing

In [ ]:
bbc_df.head()

In [ ]:
bbc_df['News_text'][2]

* Lower casing

In [ ]:
# Lower Casing
bbc_df['News_text']= bbc_df['News_text'].apply(lambda x: x.lower())

In [ ]:
# removing HTML tags
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re                                                                    # regular expression module
    clean = re.compile('<.*?>')                                                  # removes anything in < >
    return re.sub(clean, '', text)

In [ ]:
bbc_df['News_text'] = bbc_df['News_text'].apply(remove_html_tags)

In [ ]:
# removing url's
def remove_urls(text):
  """ Remove url tags from a string """
  url_pattern = r"https?://+|www\."                       # \S+ --> matches anything non-white space character with repetations ; ? --> matches 0 or 1 occurences of pattern to  its left 
  without_urls = re.sub(pattern=url_pattern, repl=' ', string=text)
  return without_urls

In [ ]:
bbc_df['News_text'] = bbc_df['News_text'].apply(remove_urls)

In [ ]:
bbc_df['News_text'][2]

In [ ]:
# removing non-word 
def rem_non_word(text):
  """ Remove non-word characters from the string """
  non_words = r"\s+[a-zA-Z]'\s+"                                               # \s+ --> matches space character with repetation(+) ; [a-zA-Z] --> match text string within range
  without_nw = re.sub(pattern = non_words , repl = '', string =text)
  return without_nw

In [ ]:
bbc_df['News_text'] = bbc_df['News_text'].apply(rem_non_word)

In [ ]:
bbc_df['News_text'] = [re.sub(r"\\n+", " ", i) for i in bbc_df['News_text']]

In [ ]:
bbc_df['News_text'][2]

In [ ]:
bbc_df['News_text'] = [re.sub(r"b\'+", "", i) for i in bbc_df['News_text']]

In [ ]:
bbc_df['News_text'][2]

In [ ]:
bbc_df['News_text'] = [re.sub(r"\\\'s+", "", i) for i in bbc_df['News_text']]

In [ ]:
bbc_df['News_text'][2]

* Removing punctuations

In [ ]:
#library that contains punctuation
import string
string.punctuation

In [ ]:
def remove_punctuation(text):
  """ Removes punctuations mentioned in the library """
  punctuation_free = "".join([i for i in text if i not in string.punctuation])    # joins everything except punctuations
  return punctuation_free

In [ ]:
bbc_df['News_text'] = bbc_df['News_text'].apply(lambda x:remove_punctuation(x))

In [ ]:
bbc_df['News_text'][2]

* Remove Numbers

In [ ]:
def remove_numbers(text):
  num_to_remove = r'\d+'                                                               # \d+ --> matches digits 0-9 with any repetation
  without_num = re.sub(pattern = num_to_remove , repl ="", string=text)
  return without_num

In [ ]:
bbc_df['News_text'] = bbc_df['News_text'].apply(remove_numbers)

In [ ]:
bbc_df['News_text'][2]

* Tokenization

In [ ]:
#Word tokenization function
def tokenization(text):
    tokens = re.split('\W+',text)                           # creates words as tokens
    return tokens

In [ ]:
bbc_df['tokens'] = bbc_df['News_text'].apply(tokenization)

In [ ]:
bbc_df.head()

* Removing Stop-words

In [ ]:
nltk.download('stopwords')

In [ ]:
# extracting the stopwords from nltk library
sw = stopwords.words('english')
# displaying the stopwords
np.array(sw)

In [ ]:
def stopwords(text):
    '''a function for removing the stopword'''
    # removing the stop words and lowercasing the selected words
    text = [word for word in text.split() if word not in sw]
    # joining the list of words with space separator
    return " ".join(text)

In [ ]:
bbc_df['News_wo_stopwords'] = bbc_df['News_text'].apply(stopwords)

In [ ]:
bbc_df.head()

* Lemmatization

In [ ]:
#defining the object for Lemmatization
lmt = WordNetLemmatizer()

In [ ]:
#defining the function for lemmatization
def lemmatizer(text):
  """ This function lematizes each word in the text """
  lemat_text = [lmt.lemmatize(word) for word in text.split()]
  return lemat_text

In [ ]:
bbc_df['lematized_news'] = bbc_df['News_text'].apply(lambda x:lemmatizer(x))

In [ ]:
bbc_df.head()

* Clean News

In [ ]:
bbc_df['clean_news'] = [' '.join(text) for text in bbc_df['lematized_news']] 

In [ ]:
bbc_df['clean_news'][2]

* POS Tagging

In [ ]:
# using spacy library

def pos_tagging(text):
  allowed_postags = ['NOUN', 'ADJ']
  tag_txt = []
  doc = nlp(text)
  tag_txt.append([token.text for token in doc if token.pos_ in allowed_postags])   
  return ' '.join(tag_txt[0])


In [ ]:
nlp = spacy.load("en_core_web_sm" ,disable=['parser', 'ner'])
bbc_df['news'] = bbc_df['clean_news'].apply(lambda x:pos_tagging(x))

In [ ]:
bbc_df.head()

In [ ]:
bbc_df['news'][0]

### Top Words

In [ ]:
# Define helper functions
def get_top_n_words(n_top_words, count_vectorizer, text_data):
    '''
    returns a tuple of the top n words in a sample and their 
    accompanying counts, given a CountVectorizer object and text sample
    '''
    vectorized_headlines = count_vectorizer.fit_transform(text_data.values)        # .values --> creates a numpy array
    vectorized_total = np.sum(vectorized_headlines, axis=0)
    word_indices = np.flip(np.argsort(vectorized_total)[0,:], 1)           # index / position of each word in all documents
    word_values = np.flip(np.sort(vectorized_total)[0,:],1)                # values of words at that position
    
    word_vectors = np.zeros((n_top_words, vectorized_headlines.shape[1]))         # n top words binary matrix for all the headlines
    for i in range(n_top_words):
        word_vectors[i,word_indices[0,i]] = 1

    words = [word[0].encode('ascii').decode('utf-8') for                           # n top words 
             word in count_vectorizer.inverse_transform(word_vectors)]

    return (words, word_values[0,:n_top_words].tolist()[0])

In [ ]:
count_vectorizer = CountVectorizer(stop_words='english')

In [ ]:
count_vectorizer = CountVectorizer(stop_words='english')
words, word_values = get_top_n_words(n_top_words=15,
                                     count_vectorizer=count_vectorizer, 
                                     text_data=bbc_df['news'])

In [ ]:
words

In [ ]:
count_vectorizer = CountVectorizer(stop_words='english')
words, word_values = get_top_n_words(n_top_words=15,
                                     count_vectorizer=count_vectorizer, 
                                     text_data=bbc_df['news'])

fig, ax = plt.subplots(figsize=(16,8))
ax.bar(range(len(words)), word_values)
ax.set_xticks(range(len(words)))
ax.set_xticklabels(words, rotation='vertical')
ax.set_title('Top words in headlines dataset (excluding stop words)', size = 15)
ax.set_xlabel('Word', size =12 )
ax.set_ylabel('Number of occurences', size=12)
plt.show()

### Word-Cloud

In [ ]:
plt.figure(figsize=(15,10))
cl_words = ' '.join(bbc_df['news'])
wordCloud = WordCloud(width=800, height=500, background_color="black", max_font_size=100).generate(cl_words)
plt.imshow(wordCloud, interpolation="bilinear", cmap = 'BuPu')
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
allWords = ' '.join([topic for topic in bbc_df[bbc_df['type']=='business']['news']])
wordCloud = WordCloud(width=500, height=300, background_color="black", random_state=21, max_font_size=100).generate(allWords)
plt.imshow(wordCloud, interpolation="bilinear", cmap = 'Greys')
plt.axis('off')
plt.show()

### Text-Vectorization

* Count Vectorization

In [ ]:
count_vectorizer = CountVectorizer(stop_words='english', max_features=4000)                         ## Common for all models
count_data = count_vectorizer.fit_transform(bbc_df['news'])
feature_names = count_vectorizer.get_feature_names()
number_topics = 5
top_words = 15

In [ ]:
print('Headline after vectorization : \n{}'.format(feature_names))

## ML Model Implementation

 ### 1. Latent Dirichlet Allocation (LDA)  



In [ ]:
# Parameter tuning 
from sklearn.model_selection import GridSearchCV 
grid_params = {'n_components':range(5, 10)}

In [ ]:
# LDA model
lda = LatentDirichletAllocation()
lda_model = GridSearchCV(lda, param_grid = grid_params)
lda_model.fit(count_data)

In [ ]:
# Best LDA model
best_lda_model = lda_model.best_estimator_

print("Best LDA model's params" , lda_model.best_params_)
print("Best log likelihood Score for the LDA model",lda_model.best_score_)
print("LDA model Perplexity on train data", best_lda_model.perplexity(count_data))

In [ ]:
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()

In [ ]:
lda_panel = pyLDAvis.sklearn.prepare(best_lda_model, count_data ,count_vectorizer,mds='tsne')
lda_panel

*LDA correctly describes the topics that we most, predicted*

### 2. Latent Semantic Analysis (LSA)

* using Count vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook

In [ ]:
# document clustering for LSA
tsvd = TruncatedSVD(n_components = 5)
tsvd.fit(count_data)
tsvd_mat = tsvd.transform(count_data)

In [ ]:
s_list =[]

for clus in tqdm(range(2, 21)):
  km = KMeans(n_clusters=clus, n_init=50, max_iter=1000)                         # Instantiate KMeans clustering
  km.fit(tsvd_mat)                                                               # Run KMeans clustering
  s = silhouette_score(tsvd_mat, km.labels_)
  s_list.append(s)

In [ ]:
plt.plot(range(2,21), s_list)
plt.show()

In [ ]:
# TSNE plot
tsne = TSNE(n_components=2)
tsne_mat = tsne.fit_transform(tsvd_mat)

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(tsne_mat[:,0],tsne_mat[:,1],hue=bbc_df['type'])

In [ ]:
def create_top_words_dict(model, features , n_top_words):                        # Defining function for top words
    """ This function gives top words."""
    top_words_dict = {}
    for topic_id, topic in enumerate(model.components_):
        top_words_dict[topic_id] = [features[i] for i in topic.argsort()[:-n_top_words - 1:-1]]        
    return top_words_dict

In [ ]:
print("Top 20 words bagged by SVD model using CountVectorizer")

svd_top_words = create_top_words_dict(tsvd, feature_names , top_words)

print(svd_top_words)

In [ ]:
pd.set_option('display.max_colwidth', None)    

In [ ]:
svd_top_words = create_top_words_dict(tsvd, feature_names , top_words)
svd_df = pd.DataFrame([svd_top_words]).T
svd_df.rename(columns = {0: 'Top 20 words'}, inplace = True)
svd_df.rename(index = {0: 'Topic 1', 1: 'Topic 2' , 2: 'Topic 3' , 3: 'Topic 4' , 4 : 'Topic 5'}, inplace = True)
svd_df

* Using TF-IDF 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.8, max_features=8000,min_df=0.05)
tfidf_matrix = vectorizer.fit_transform(bbc_df['news'])

In [ ]:
tfidf_matrix.shape

In [ ]:
# svd model

svd_model = TruncatedSVD(n_components = 5, algorithm ='randomized', n_iter =100, random_state = 0)
svd_model.fit(tfidf_matrix)

In [ ]:
terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:15]
    print("Topic \n" +str(i)+" " )
    for t in sorted_terms:
        print(t[0],end=" ")


### 3. LDA using Gensim library

In [ ]:
import gensim
from gensim import corpora

In [ ]:
# creating word vector for gensim 
dtm_g = bbc_df['news'].str.split().tolist()

In [ ]:
dtm_g[:1]

In [ ]:
id2word = corpora.Dictionary(dtm_g)                                                # mapping words to tokens
# counting the number of occurrences of each distinct word,--> converting to its integer word id and return the result as a sparse vector.
corpus = [id2word.doc2bow(text) for text in dtm_g]                                

In [ ]:
print(corpus[:120])

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [ ]:
lda_g = gensim.models.ldamodel.LdaModel                                           # creating object for lda using gensim library
lda_model = lda_g(corpus = corpus, num_topics = 5, id2word=id2word, random_state= 101, chunksize = 500 , passes=10 , eval_every =None )

In [ ]:
lda_model.print_topics()

*The above output means: each of the unique words are given weights based on the topics.This implies which of the words dominate the topics.*

In [ ]:
print(lda_model.print_topics(num_topics=6, num_words=5))

In [ ]:
import pyLDAvis.gensim_models

In [ ]:
pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim_models.prepare(lda_model,corpus,id2word)
vis

In [ ]:
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=dtm_g, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

### **Conclusion**



* Overall 3 different algorithms were used for evaluation of topic modelling.
* LDA provided considerable result , with distinct topics as expected.
* LSA with CountVectorizer gave somoewhat considerable results, but with TF-IDF the results were not as expected.
* LDA using gensim library provided the best result with a considerable coherence score of 0.52 
* Topics represented in LDA were adjacent, with hidden topics and relationship between words and documents were found with multiple probability distribution
